# Tópico 09 – Merge [<img src="images/colag_logo.svg" style="float: right; vertical-align: middle; width: 42px; height: 42px;">](https://colab.research.google.com/github/urielmoreirasilva/urielmoreirasilva.github.io/blob/main/aulas/T%C3%B3pico%2009/09%20%E2%80%93%20Merge.ipynb) [<img src="images/github_logo.svg" style="float: right; margin-right: 12px; vertical-align: middle; width: 36px; height: 36px;">](https://github.com/urielmoreirasilva/urielmoreirasilva.github.io/blob/main/aulas/T%C3%B3pico%2009/09%20%E2%80%93%20Merge.ipynb)

Vamos aprender como juntar DataFrames diferentes!

### Resultados Esperados
1. Aprender um pouco sobre o uso do `merge`.

### Referências
- [BPD, Capítulos 11 e 13](https://notes.dsc10.com/)

Material adaptado do [DSC10 (UCSD)](https://dsc10.com/) por [Flavio Figueiredo (DCC-UFMG)](https://flaviovdf.io/fcd/) e [Uriel Silva (DEST-UFMG)](https://urielmoreirasilva.github.io)

In [1]:
# Importando BabyPandas, Numpy e o Pandas
import numpy as np
import babypandas as bpd

## Merge

<center>
    <img src = "images/FusionDanceFinaleGotenTrunksBuuSaga.webp" width = 45%>    
</center>

### Motivação

Considere uma situação em que temos 2 `DataFrame`s diferentes:
- `telefones`, um DataFrame contendo informações gerais e especificações técnicas acerca de vários modelos de smartphones;
- `unidades`, um DataFrame contendo o número de unidades por loja de eletrônicos da franquia da qual você é dono, em cada um dos shoppings de Belo Horizonte.

In [2]:
# DataFrame 1
telefones = bpd.DataFrame().assign(
    Modelo=['iPhone 13', 'iPhone 13 Pro Max', 'Samsung Galaxy Z Flip', 'Pixel 5a'],
    Preco=[799, 1099, 999, 449],
    Tela=[6.1, 6.7, 6.7, 6.3]
)

# DataFrame 2
unidades = bpd.DataFrame().assign(
    Celular=['iPhone 13 Pro Max', 'iPhone 13', 'Pixel 5a', 'iPhone 13'],
    Unidades=[50, 40, 10, 100],
    Shopping=['Del Rey', 'Savassi', 'Diamond', 'Cidade']
)

In [3]:
# DataFrame1
telefones

,Modelo,Preco,Tela
0,iPhone 13,799,6.1
1,iPhone 13 Pro Max,1099,6.7
2,Samsung Galaxy Z Flip,999,6.7
3,Pixel 5a,449,6.3


In [4]:
# DataFrame2
unidades

,Celular,Unidades,Shopping
0,iPhone 13 Pro Max,50,Del Rey
1,iPhone 13,40,Savassi
2,Pixel 5a,10,Diamond
3,iPhone 13,100,Cidade


**Pergunta:** Se você vender todos os telefones do seu estoque (isto é, todas as unidades de todas as lojas), qual será a sua receita total? 🤔

Para responder essa pergunta, primeiramente precisamos _cruzar_ as informações do preço de mercado e da quantidade de cada celular que temos disponíveis em nossas lojas.

Fazemos isso através do método `.merge`!

In [5]:
telefones.merge(unidades, left_on = 'Modelo', right_on = 'Celular')

,Modelo,Preco,Tela,Celular,Unidades,Shopping
0,iPhone 13,799,6.1,iPhone 13,40,Savassi
1,iPhone 13,799,6.1,iPhone 13,100,Cidade
2,iPhone 13 Pro Max,1099,6.7,iPhone 13 Pro Max,50,Del Rey
3,Pixel 5a,449,6.3,Pixel 5a,10,Diamond


Mas o que acabou de acontecer?! 🤯

### O método `.merge`

O método `.merge` funciona da seguinte maneira:
- Escolha um DataFrame "à esquerda" (`left_df`) e outro "à direita" (`right_df`).
- Escolha uma coluna de cada um desses DataFrames, isto é, `left_column_name` e `right_column_name`, respectivamente.
- Execute o merge invocando:
```python
left_df.merge(
    right_df, 
    left_on = left_column_name,
    right_on = right_column_name
)
```

- O DataFrame resultante de um `.merge` contém uma única linha para cada correspondência entre as duas colunas.
- _Usualmente_, os argumentos `left_on` e `right_on` são iguais, mas como no exemplo acima, _isso não é necessário_.
- Note que as linhas em qualquer um dos DataFrames sem correspondência com o outro desaparecem após o merge!

Voltando à nossa pergunta original, para calcular a receita total da venda de todos os aparelhos em todas as lojas, basta somar os produtos entre as colunas `Preco` e `Unidades` após realizarmos o merge:

In [6]:
# Merge: DataFrames 1 e 2, por `Modelo` e `Celular`
merge = telefones.merge(
    unidades,
    left_on = 'Modelo',
    right_on = 'Celular'
)

In [7]:
# Calculando a receita total no DataFrame sobre o qual fizemos o _merge_ 
(merge.get('Preco') * merge.get('Unidades')).sum()

171300

Note que a ordem das colunas escolhidas para o merge _não influencia_ o resultado final, apenas a _ordenação_ das colunas do DataFrame resultante.

In [8]:
unidades.merge(telefones, left_on = 'Celular', right_on = 'Modelo')

,Celular,Unidades,Shopping,Modelo,Preco,Tela
0,iPhone 13 Pro Max,50,Del Rey,iPhone 13 Pro Max,1099,6.7
1,iPhone 13,40,Savassi,iPhone 13,799,6.1
2,iPhone 13,100,Cidade,iPhone 13,799,6.1
3,Pixel 5a,10,Diamond,Pixel 5a,449,6.3


### `.merge` com índices

- O método `.merge` pode ser utilizado com um `DataFrame` _indexado_, e nesse caso toma uma forma ainda mais simples. 
- Nesse caso, em vez de especificarmos ambos `left_on` e `right_on`, especificamos apenas `left_index = True` ou `right_index = True`.

In [9]:
telefones

,Modelo,Preco,Tela
0,iPhone 13,799,6.1
1,iPhone 13 Pro Max,1099,6.7
2,Samsung Galaxy Z Flip,999,6.7
3,Pixel 5a,449,6.3


In [10]:
unidades_com_index = unidades.set_index('Celular')
unidades_com_index

,Unidades,Shopping
Celular,,
iPhone 13 Pro Max,50,Del Rey
iPhone 13,40,Savassi
Pixel 5a,10,Diamond
iPhone 13,100,Cidade


In [11]:
telefones.merge(
    unidades_com_index,
    left_on = 'Modelo',
    right_index = True
)

,Modelo,Preco,Tela,Unidades,Shopping
0,iPhone 13,799,6.1,40,Savassi
0,iPhone 13,799,6.1,100,Cidade
1,iPhone 13 Pro Max,1099,6.7,50,Del Rey
3,Pixel 5a,449,6.3,10,Diamond


Como dito anteriormente, obtemos os mesmos resultados se indexarmos o DataFrame `telefones`, apesar de isso não ser necessário:

In [12]:
telefones_com_index = telefones.set_index('Modelo')
telefones_com_index.merge(
    unidades_com_index,
    left_on = 'Modelo',
    right_index = True
)

,Preco,Tela,Unidades,Shopping
Modelo,,,,
iPhone 13,799,6.1,40,Savassi
iPhone 13,799,6.1,100,Cidade
iPhone 13 Pro Max,1099,6.7,50,Del Rey
Pixel 5a,449,6.3,10,Diamond


... e vice-versa com `right_on` e `left_index = True`:

In [13]:
unidades_com_index.merge(
    telefones_com_index,
    right_on = 'Modelo',
    left_index = True
)

,Unidades,Shopping,Preco,Tela
Modelo,,,,
iPhone 13 Pro Max,50,Del Rey,1099,6.7
iPhone 13,40,Savassi,799,6.1
iPhone 13,100,Cidade,799,6.1
Pixel 5a,10,Diamond,449,6.3


### `.merge` com rótulos iguais

- Finalmente, note que se ambos `left_column_name` e `right_column_name` forem iguais, podemos especificar o `.merge` apenas com a opção `on`.

In [14]:
# Criando uma nova coluna `Modelo` igual a `Celular`
unidades_modelo = unidades.assign(Modelo = unidades.get('Celular'))

# Deletando a coluna `Celular`
unidades_modelo = unidades_modelo.drop(columns = 'Celular')

unidades_modelo

,Unidades,Shopping,Modelo
0,50,Del Rey,iPhone 13 Pro Max
1,40,Savassi,iPhone 13
2,10,Diamond,Pixel 5a
3,100,Cidade,iPhone 13


### Um outro exemplo: "cidades com clima bom" e "cidades com universidades"

Considere o seguinte exemplo, com 2 DataFrames: um contendo algumas "cidades com clima bom" e outro contendo algumas "cidades com universidades", em diferentes estados dos EUA. 

In [15]:
# Cidades com clima bom
nice_weather_cities = bpd.DataFrame().assign(
    city=['La Jolla', 'San Diego', 'Austin', 'Los Angeles'],
    today_high_temp=['79', '83', '87', '87']
    
)

nice_weather_cities

,city,today_high_temp
0,La Jolla,79
1,San Diego,83
2,Austin,87
3,Los Angeles,87


In [16]:
# Cidades com escolas
schools = bpd.DataFrame().assign(
    name=['UCSD', 'University of Chicago', 'University of San Diego','Johns Hopkins University', 'UT Austin', 'SDSU', 'UCLA'], 
    city=['La Jolla', 'Chicago', 'San Diego', 'Baltimore', 'Austin', 'San Diego', 'Los Angeles'],
    state=['California', 'Illinois', 'California', 'Maryland', 'Texas', 'California', 'California'],
    graduation_rate=[0.87, 0.94, 0.78, 0.92, 0.81, 0.83, 0.91 ]
)

schools

,name,city,state,graduation_rate
0,UCSD,La Jolla,California,0.87
1,University of Chicago,Chicago,Illinois,0.94
2,University of San Diego,San Diego,California,0.78
3,Johns Hopkins University,Baltimore,Maryland,0.92
4,UT Austin,Austin,Texas,0.81
5,SDSU,San Diego,California,0.83
6,UCLA,Los Angeles,California,0.91


### Exercício ✅

Apenas olhando as células acima e **sem escrever nenhuma linha de código**, quantas linhas terá o DataFrame `nice_weather_cities.merge(schools, on = 'city')`?

A. 4

B. 5

C. 6

D. 7

E. 8

## Resumo

- Para combinar informações de vários `DataFrame`s, podemos utilizar o método `.merge`.
- Ao usar `.merge`, o Python procura uma correspondência entre uma coluna especificada em cada DataFrame, e combina as linhas onde há correspondência.
- Nos casos em que não houver correspondência, a linha desaparece!